# Tutorial 02-FF — Train a SOEN Model with Forward-Forward Algorithm

This tutorial demonstrates training a SOEN model using the **Forward-Forward (FF) algorithm** (Hinton, 2022) instead of backpropagation.

## Why Forward-Forward for SOEN?

The FF algorithm is remarkably well-suited for SOEN hardware because:
- **No backward pass needed** — Only forward passes through the network
- **Local objectives** — Each layer learns independently
- **Designed for analog hardware** — Works with unknown non-linearities
- **Potential for on-chip learning** — Could enable hardware-based training

## Hardware/Software Split

**IMPORTANT**: This notebook uses FF as a SOFTWARE-FLEXIBLE training method.
The HARDWARE-FIXED SOEN dynamics (`ds/dt = γ⁺g(φ) - γ⁻s`) remain unchanged.

| Component | Classification |
|-----------|----------------|
| SOEN dynamics (ODE) | 🔒 Hardware-Fixed |
| Source function g(φ) | 🔒 Hardware-Fixed |
| Goodness function (Σs²) | 🟢 Software-Flexible |
| Layer normalization | 🟢 Software-Flexible |
| FF training loop | 🟢 Software-Flexible |

## ML Task Overview

Same as Tutorial 02: Binary classification on time-series inputs:
- Class 0: Input contains a single pulse
- Class 1: Input contains two distinct pulses

## 1. Imports and Setup

In [ ]:
# Setup: Ensure soen_toolkit is importable
import sys
from pathlib import Path

# Add src directory to path if running from notebook location
notebook_dir = Path.cwd()
for parent in [notebook_dir] + list(notebook_dir.parents):
    candidate = parent / "src"
    if (candidate / "soen_toolkit").exists():
        sys.path.insert(0, str(candidate))
        break

from typing import Dict, List, Tuple, Optional
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tqdm.auto import tqdm

# HARDWARE-FIXED: Import existing SOEN model (do NOT modify these)
from soen_toolkit.core import SOENModelCore
from soen_toolkit.core.model_yaml import build_model_from_yaml

# Set random seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print("Setup complete!")

## 2. Forward-Forward Algorithm Components (Software-Flexible)

These are the FF-specific components that we ADD to the existing SOEN framework.
They do NOT modify any hardware-fixed code.

In [ ]:
# ============================================================================
# SOFTWARE-FLEXIBLE: Forward-Forward Components
# These are training constructs, not physics.
# ============================================================================

class GoodnessFunction(nn.Module):
    """
    Compute goodness from SOEN layer states.
    
    Goodness = sum of squared activities (Hinton, 2022)
    This is a SOFTWARE-FLEXIBLE training objective.
    """
    
    def __init__(self, threshold: float = 2.0):
        super().__init__()
        self.threshold = threshold
    
    def forward(self, states: torch.Tensor) -> torch.Tensor:
        """
        Compute goodness from layer states.
        
        Args:
            states: [batch, time, neurons] or [batch, neurons]
        
        Returns:
            goodness: [batch] - sum of squared activities
        """
        # If we have time dimension, use final timestep
        if states.dim() == 3:
            states = states[:, -1, :]  # [batch, neurons]
        
        # Goodness = sum of squared states
        goodness = torch.sum(states ** 2, dim=-1)  # [batch]
        return goodness
    
    def loss(self, goodness: torch.Tensor, is_positive: torch.Tensor) -> torch.Tensor:
        """
        FF loss: maximize goodness for positive, minimize for negative.
        
        Args:
            goodness: [batch] - computed goodness values
            is_positive: [batch] - 1.0 for positive samples, 0.0 for negative
        
        Returns:
            loss: scalar BCE loss
        """
        # p(positive) = sigmoid(goodness - threshold)
        logits = goodness - self.threshold
        loss = F.binary_cross_entropy_with_logits(logits, is_positive)
        return loss


class FFLayerNorm(nn.Module):
    """
    Layer normalization for Forward-Forward algorithm.
    
    Normalizes to unit length, removing goodness information
    while preserving orientation (relative activities).
    
    This is a SOFTWARE-FLEXIBLE training technique.
    """
    
    def __init__(self, eps: float = 1e-8):
        super().__init__()
        self.eps = eps
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Normalize input to unit length.
        
        Args:
            x: [batch, neurons] - layer activations
        
        Returns:
            normalized: [batch, neurons] - unit length vectors
        """
        norm = torch.sqrt(torch.sum(x ** 2, dim=-1, keepdim=True) + self.eps)
        return x / norm


print("FF components defined (Software-Flexible)")

## 3. Load Data

Load the same pulse classification dataset used in Tutorial 02.

In [ ]:
# Path to dataset
DATA_PATH = Path("training/datasets/soen_seq_task_one_or_two_pulses_seq64.hdf5")

def load_pulse_dataset(path: Path, split: str = "train") -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Load pulse classification dataset.
    
    Args:
        path: Path to HDF5 file
        split: 'train', 'val', or 'test'
    
    Returns:
        data: [N, T, D] input sequences
        labels: [N] class labels (0 or 1)
    """
    with h5py.File(path, 'r') as f:
        data = torch.tensor(f[split]['data'][:], dtype=torch.float32)
        labels = torch.tensor(f[split]['labels'][:], dtype=torch.long)
    return data, labels

# Load datasets
if DATA_PATH.exists():
    train_data, train_labels = load_pulse_dataset(DATA_PATH, 'train')
    val_data, val_labels = load_pulse_dataset(DATA_PATH, 'val')
    
    print(f"Train: {train_data.shape}, labels: {train_labels.shape}")
    print(f"Val: {val_data.shape}, labels: {val_labels.shape}")
    print(f"Unique labels: {torch.unique(train_labels).tolist()}")
else:
    print(f"Dataset not found at {DATA_PATH}")
    print("Please ensure the dataset exists or update the path.")

## 4. Build SOEN Model (Hardware-Fixed)

Load the same model architecture used in Tutorial 02.
**The model dynamics are HARDWARE-FIXED and unchanged.**

In [ ]:
# ============================================================================
# HARDWARE-FIXED: Load existing SOEN model
# We use the model AS-IS, only adding FF training on top
# ============================================================================

MODEL_PATH = Path("training/test_models/model_specs/1D_5D_2D_PulseNetSpec.yaml")

if MODEL_PATH.exists():
    # Load the model - this uses hardware-fixed SOEN dynamics
    model = build_model_from_yaml(MODEL_PATH)
    
    # Update dt to match training config
    model.dt = 779
    
    print("Model Architecture:")
    print(f"  Layers: {len(model.layers)}")
    for i, layer in enumerate(model.layers):
        print(f"    Layer {i}: {type(layer).__name__}")
    print(f"  Connections: {list(model.connections.keys())}")
    
    # The model uses these hardware-fixed components:
    # - SingleDendrite dynamics: ds/dt = gamma_plus * g(phi) - gamma_minus * s
    # - Source function g(phi): Heaviside_fit_state_dep
    # These are NOT modified by FF training
else:
    print(f"Model spec not found at {MODEL_PATH}")

## 5. Forward-Forward Training Implementation

This is the core FF training logic. Key points:
1. **Embed label into input** - Following Hinton's supervised FF approach
2. **Positive pass** - Run with correct label, maximize goodness
3. **Negative pass** - Run with wrong label, minimize goodness
4. **Layer-wise learning** - Each layer learns independently

**CRITICAL**: We WRAP the existing SOEN model, we don't modify its dynamics.

In [ ]:
class FFSOENTrainer:
    """
    Forward-Forward trainer for SOEN models.
    
    This trainer:
    - WRAPS the existing SOEN model (hardware-fixed dynamics unchanged)
    - ADDS FF-specific components (goodness, layer norm) for training
    - Uses two forward passes instead of backprop
    """
    
    def __init__(
        self,
        model: SOENModelCore,
        num_classes: int = 2,
        goodness_threshold: float = 2.0,
        learning_rate: float = 0.001,
        device: str = "cpu",
    ):
        self.model = model.to(device)
        self.num_classes = num_classes
        self.device = device
        
        # SOFTWARE-FLEXIBLE: FF components
        self.goodness_fn = GoodnessFunction(threshold=goodness_threshold)
        self.layer_norm = FFLayerNorm()
        
        # Optimizer for model weights
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        # Track metrics
        self.train_losses = []
        self.val_accuracies = []
    
    def create_ff_input(
        self,
        data: torch.Tensor,
        labels: torch.Tensor,
        use_correct_label: bool = True,
    ) -> torch.Tensor:
        """
        Create FF input by embedding label into the input signal.
        
        For this task, we scale the input based on the label:
        - This embeds label information into the input magnitude
        - Positive data: input with correct label scaling
        - Negative data: input with wrong label scaling
        
        Args:
            data: [batch, time, 1] - raw input signals
            labels: [batch] - true class labels
            use_correct_label: True for positive pass, False for negative
        
        Returns:
            ff_input: [batch, time, 1] - input with embedded label
        """
        batch_size = data.shape[0]
        
        if use_correct_label:
            # Positive: use true labels
            label_scale = labels.float()
        else:
            # Negative: use wrong labels (flip 0<->1 for binary)
            label_scale = 1.0 - labels.float()
        
        # Embed label as a bias added to the input
        # This is a simple embedding; more sophisticated methods exist
        label_bias = label_scale.view(batch_size, 1, 1) * 0.1
        ff_input = data + label_bias
        
        return ff_input
    
    def forward_pass(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Run forward pass through SOEN model.
        
        HARDWARE-FIXED: Uses existing model dynamics (unchanged)
        SOFTWARE-FLEXIBLE: Extracts states for goodness computation
        
        Args:
            x: [batch, time, input_dim] - input sequences
        
        Returns:
            hidden_states: [batch, neurons] - final hidden layer states
            output: [batch, output_dim] - model output
        """
        # Run through existing SOEN model (HARDWARE-FIXED dynamics)
        self.model.set_tracking(track_s=True)  # Track states for goodness
        
        with torch.enable_grad():
            output, all_histories = self.model(x)
        
        # Extract hidden layer states (layer 1 = SingleDendrite)
        # all_histories[0] = input layer, all_histories[1] = hidden layer
        if len(all_histories) > 1:
            hidden_states = all_histories[1][:, -1, :]  # Final timestep
        else:
            hidden_states = output[:, -1, :] if output.dim() == 3 else output
        
        return hidden_states, output
    
    def train_step(
        self,
        data: torch.Tensor,
        labels: torch.Tensor,
    ) -> Dict[str, float]:
        """
        One FF training step.
        
        1. Positive pass: input with correct label -> maximize goodness
        2. Negative pass: input with wrong label -> minimize goodness
        
        Args:
            data: [batch, time, 1] - input sequences
            labels: [batch] - class labels
        
        Returns:
            metrics: dict with loss values
        """
        self.model.train()
        self.optimizer.zero_grad()
        
        batch_size = data.shape[0]
        
        # === POSITIVE PASS ===
        pos_input = self.create_ff_input(data, labels, use_correct_label=True)
        pos_hidden, _ = self.forward_pass(pos_input)
        pos_goodness = self.goodness_fn(pos_hidden)
        
        # === NEGATIVE PASS ===
        neg_input = self.create_ff_input(data, labels, use_correct_label=False)
        neg_hidden, _ = self.forward_pass(neg_input)
        neg_goodness = self.goodness_fn(neg_hidden)
        
        # === COMPUTE FF LOSS ===
        # Combine positive and negative samples
        all_goodness = torch.cat([pos_goodness, neg_goodness], dim=0)
        is_positive = torch.cat([
            torch.ones(batch_size, device=self.device),
            torch.zeros(batch_size, device=self.device)
        ], dim=0)
        
        loss = self.goodness_fn.loss(all_goodness, is_positive)
        
        # === BACKWARD (through goodness, not through layers) ===
        loss.backward()
        self.optimizer.step()
        
        return {
            "loss": loss.item(),
            "pos_goodness": pos_goodness.mean().item(),
            "neg_goodness": neg_goodness.mean().item(),
        }
    
    def evaluate(
        self,
        data: torch.Tensor,
        labels: torch.Tensor,
    ) -> float:
        """
        Evaluate using FF classification strategy.
        
        For each sample, run with each possible label and
        pick the label that gives highest goodness.
        
        Args:
            data: [batch, time, 1] - input sequences
            labels: [batch] - true labels
        
        Returns:
            accuracy: float
        """
        self.model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for i in range(len(data)):
                sample = data[i:i+1]  # [1, time, 1]
                true_label = labels[i].item()
                
                # Test each possible label
                goodness_per_label = []
                for test_label in range(self.num_classes):
                    test_labels = torch.tensor([test_label], device=self.device)
                    ff_input = self.create_ff_input(
                        sample,
                        test_labels,
                        use_correct_label=True  # Treat as if this is the "correct" label
                    )
                    hidden, _ = self.forward_pass(ff_input)
                    goodness = self.goodness_fn(hidden).item()
                    goodness_per_label.append(goodness)
                
                # Predict label with highest goodness
                predicted = np.argmax(goodness_per_label)
                
                if predicted == true_label:
                    correct += 1
                total += 1
        
        return correct / total if total > 0 else 0.0


print("FFSOENTrainer defined")

## 6. Training Loop

In [ ]:
# Training configuration
CONFIG = {
    "batch_size": 32,
    "num_epochs": 20,
    "learning_rate": 0.001,
    "goodness_threshold": 2.0,
    "device": "cpu",
}

def create_dataloader(data, labels, batch_size, shuffle=True):
    """Simple dataloader generator."""
    dataset_size = len(data)
    indices = list(range(dataset_size))
    
    if shuffle:
        random.shuffle(indices)
    
    for start_idx in range(0, dataset_size, batch_size):
        end_idx = min(start_idx + batch_size, dataset_size)
        batch_indices = indices[start_idx:end_idx]
        yield data[batch_indices], labels[batch_indices]

print(f"Training config: {CONFIG}")

In [ ]:
# Initialize trainer (only if data and model are available)
if 'model' in dir() and 'train_data' in dir():
    # Rebuild model for fresh training
    model = build_model_from_yaml(MODEL_PATH)
    model.dt = 779
    
    trainer = FFSOENTrainer(
        model=model,
        num_classes=2,
        goodness_threshold=CONFIG["goodness_threshold"],
        learning_rate=CONFIG["learning_rate"],
        device=CONFIG["device"],
    )
    
    # Move data to device
    train_data_device = train_data.to(CONFIG["device"])
    train_labels_device = train_labels.to(CONFIG["device"])
    val_data_device = val_data.to(CONFIG["device"])
    val_labels_device = val_labels.to(CONFIG["device"])
    
    print("Trainer initialized")
    print(f"Training samples: {len(train_data)}")
    print(f"Validation samples: {len(val_data)}")
else:
    print("Model or data not available. Please run previous cells.")

In [ ]:
# Training loop
if 'trainer' in dir():
    print("Starting Forward-Forward Training")
    print("=" * 60)
    print("Note: FF uses local objectives per layer, not global backprop.")
    print("The SOEN dynamics (ds/dt = gamma+ * g(phi) - gamma- * s) are UNCHANGED.")
    print("=" * 60)
    
    history = {
        "train_loss": [],
        "pos_goodness": [],
        "neg_goodness": [],
        "val_accuracy": [],
    }
    
    for epoch in range(CONFIG["num_epochs"]):
        epoch_losses = []
        epoch_pos_goodness = []
        epoch_neg_goodness = []
        
        # Training
        for batch_data, batch_labels in create_dataloader(
            train_data_device, train_labels_device, CONFIG["batch_size"]
        ):
            metrics = trainer.train_step(batch_data, batch_labels)
            epoch_losses.append(metrics["loss"])
            epoch_pos_goodness.append(metrics["pos_goodness"])
            epoch_neg_goodness.append(metrics["neg_goodness"])
        
        # Validation (subsample for speed)
        val_subset = min(200, len(val_data_device))
        val_acc = trainer.evaluate(
            val_data_device[:val_subset],
            val_labels_device[:val_subset]
        )
        
        # Record history
        avg_loss = np.mean(epoch_losses)
        avg_pos = np.mean(epoch_pos_goodness)
        avg_neg = np.mean(epoch_neg_goodness)
        
        history["train_loss"].append(avg_loss)
        history["pos_goodness"].append(avg_pos)
        history["neg_goodness"].append(avg_neg)
        history["val_accuracy"].append(val_acc)
        
        print(f"Epoch {epoch+1}/{CONFIG['num_epochs']} | "
              f"Loss: {avg_loss:.4f} | "
              f"Pos G: {avg_pos:.2f} | "
              f"Neg G: {avg_neg:.2f} | "
              f"Val Acc: {val_acc*100:.1f}%")
    
    print("\nTraining complete!")
else:
    print("Trainer not initialized. Please run previous cells.")

## 7. Visualize Training Results

In [ ]:
if 'history' in dir() and len(history["train_loss"]) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Plot 1: Loss
    ax1 = axes[0]
    ax1.plot(history["train_loss"], 'b-', linewidth=2)
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("FF Loss")
    ax1.set_title("Training Loss")
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Goodness
    ax2 = axes[1]
    ax2.plot(history["pos_goodness"], 'g-', linewidth=2, label="Positive")
    ax2.plot(history["neg_goodness"], 'r-', linewidth=2, label="Negative")
    ax2.axhline(y=CONFIG["goodness_threshold"], color='k', linestyle='--', 
                label=f"Threshold ({CONFIG['goodness_threshold']})")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Goodness")
    ax2.set_title("Goodness (Positive vs Negative)")
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Accuracy
    ax3 = axes[2]
    ax3.plot([a * 100 for a in history["val_accuracy"]], 'purple', linewidth=2)
    ax3.set_xlabel("Epoch")
    ax3.set_ylabel("Accuracy (%)")
    ax3.set_title("Validation Accuracy")
    ax3.set_ylim([0, 100])
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final results
    print("\n" + "="*60)
    print("FINAL RESULTS")
    print("="*60)
    print(f"Final Loss: {history['train_loss'][-1]:.4f}")
    print(f"Final Validation Accuracy: {history['val_accuracy'][-1]*100:.1f}%")
    print(f"Final Positive Goodness: {history['pos_goodness'][-1]:.2f}")
    print(f"Final Negative Goodness: {history['neg_goodness'][-1]:.2f}")
    print(f"Goodness Gap: {history['pos_goodness'][-1] - history['neg_goodness'][-1]:.2f}")
else:
    print("No training history available.")

In [ ]:
# ============================================================================
# DETAILED EVALUATION: Sample predictions and confusion matrix
# ============================================================================

if 'trainer' in dir() and 'val_data_device' in dir():
    from sklearn.metrics import confusion_matrix
    
    # Evaluate on full validation set (or subset for speed)
    eval_size = min(200, len(val_data_device))
    eval_data = val_data_device[:eval_size]
    eval_labels = val_labels_device[:eval_size]
    
    # Get predictions using FF strategy (highest goodness wins)
    predictions = []
    goodness_values = []
    
    trainer.model.eval()
    with torch.no_grad():
        for i in range(len(eval_data)):
            sample = eval_data[i:i+1]
            
            # Test each label
            sample_goodness = []
            for test_label in range(trainer.num_classes):
                test_labels = torch.tensor([test_label], device=trainer.device)
                ff_input = trainer.create_ff_input(sample, test_labels, use_correct_label=True)
                hidden, _ = trainer.forward_pass(ff_input)
                goodness = trainer.goodness_fn(hidden).item()
                sample_goodness.append(goodness)
            
            predictions.append(np.argmax(sample_goodness))
            goodness_values.append(sample_goodness)
    
    predictions = np.array(predictions)
    true_labels = eval_labels.cpu().numpy()
    
    # Calculate accuracy
    accuracy = (predictions == true_labels).mean() * 100
    print(f"Evaluation Accuracy: {accuracy:.1f}% ({(predictions == true_labels).sum()}/{len(predictions)})")
    
    # ---- Visualize sample predictions ----
    fig, axes = plt.subplots(2, 4, figsize=(16, 6))
    
    for i, ax in enumerate(axes.flatten()):
        if i >= len(eval_data):
            break
        
        # Plot input signal
        signal = eval_data[i, :, 0].cpu().numpy()
        ax.plot(signal, 'b-', linewidth=1.5)
        
        true_label = true_labels[i]
        pred_label = predictions[i]
        
        color = 'green' if true_label == pred_label else 'red'
        ax.set_title(f"True: {true_label}, Pred: {pred_label}\nG0={goodness_values[i][0]:.2f}, G1={goodness_values[i][1]:.2f}", 
                    color=color, fontsize=10)
        ax.set_xlabel("Time")
        ax.set_ylabel("Input")
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f"FF Sample Predictions (Accuracy: {accuracy:.1f}%)", fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # ---- Confusion Matrix ----
    cm = confusion_matrix(true_labels, predictions)
    
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(cm, cmap='Blues')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    ax.set_title('FF Confusion Matrix')
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['0 (1 pulse)', '1 (2 pulses)'])
    ax.set_yticklabels(['0 (1 pulse)', '1 (2 pulses)'])
    
    # Add text annotations
    for i in range(2):
        for j in range(2):
            ax.text(j, i, str(cm[i, j]), ha='center', va='center', 
                   color='white' if cm[i, j] > cm.max()/2 else 'black', fontsize=16)
    
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()
    
    # ---- Goodness Distribution ----
    fig, ax = plt.subplots(figsize=(10, 4))
    
    goodness_arr = np.array(goodness_values)
    
    # Split by true label
    g0_class0 = goodness_arr[true_labels == 0, 0]  # Goodness for label 0, true class 0
    g1_class0 = goodness_arr[true_labels == 0, 1]  # Goodness for label 1, true class 0
    g0_class1 = goodness_arr[true_labels == 1, 0]  # Goodness for label 0, true class 1
    g1_class1 = goodness_arr[true_labels == 1, 1]  # Goodness for label 1, true class 1
    
    x = np.arange(2)
    width = 0.35
    
    ax.bar(x - width/2, [g0_class0.mean(), g0_class1.mean()], width, label='G(label=0)', color='blue', alpha=0.7)
    ax.bar(x + width/2, [g1_class0.mean(), g1_class1.mean()], width, label='G(label=1)', color='orange', alpha=0.7)
    
    ax.axhline(y=CONFIG["goodness_threshold"], color='red', linestyle='--', label=f'Threshold ({CONFIG["goodness_threshold"]})')
    
    ax.set_xlabel('True Class')
    ax.set_ylabel('Mean Goodness')
    ax.set_title('FF Goodness by True Class and Tested Label')
    ax.set_xticks(x)
    ax.set_xticklabels(['Class 0 (1 pulse)', 'Class 1 (2 pulses)'])
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation:")
    print("- For correct classification: G(correct_label) > G(wrong_label)")
    print("- Class 0 samples should have higher G(label=0)")
    print("- Class 1 samples should have higher G(label=1)")
else:
    print("Trainer not available. Run training first.")

## 8. Comparison: FF vs Backpropagation

| Aspect | Backpropagation (Tutorial 02) | Forward-Forward (This Tutorial) |
|--------|------------------------------|--------------------------------|
| **Training Algorithm** | Global loss + backward pass | Local goodness + two forward passes |
| **Gradient Computation** | Requires surrogate gradients | No gradients through spike |
| **Hardware Compatibility** | Low (needs backprop circuit) | High (forward passes only) |
| **SOEN Dynamics** | Unchanged | Unchanged |
| **On-Chip Learning** | Not possible | Potentially possible |

## 9. Key Takeaways

### Hardware/Software Split Preserved

1. **HARDWARE-FIXED (Unchanged)**:
   - Dendritic dynamics: `ds/dt = γ⁺·g(φ) - γ⁻·s`
   - Source function: `g(φ)` lookup
   - Spike mechanism
   - Physical constants

2. **SOFTWARE-FLEXIBLE (New for FF)**:
   - Goodness function: `Σs²`
   - Layer normalization
   - Positive/negative data generation
   - FF training loop

### FF Advantages for SOEN

- No backpropagation through complex g(φ)
- Local learning per layer
- Could enable on-chip learning
- Works with unknown non-linearities

### Next Steps

- Tune goodness threshold for your task
- Try different label embedding strategies
- Compare accuracy with backprop baseline
- Explore FF for larger SOEN networks

---

## References

- Hinton, G. (2022). The Forward-Forward Algorithm: Some Preliminary Investigations. arXiv:2212.13345
- See `reports/forward_forward_soen_analysis.md` for detailed analysis
- Compare with `02_train_a_model.ipynb` for backpropagation approach